In [1]:
import numpy as np
import pandas as pd

# base model

In [1]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda:1"

base_model_name = "LDCC/LDCC-SOLAR-10.7B"

model = AutoModelForCausalLM.from_pretrained(base_model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(base_model_name, revision='v1.1')

/home/seungbinyang/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/seungbinyang/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/seungbinyang/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Loading checkpoint shards: 100%|██████████| 5/5 [00:03<00:00,  1.33it/s]
tokenizer_config.json: 100%|██████████| 1.33k/1.33k [00:00<00:00, 4.53MB/s]
tokenizer.json: 100%|██████████| 2.

In [3]:
messages = [
    {"role": "user", "content": "대한민국의 수도는 어디야?"},
    {"role": "assistant", "content": "대한민국의 수도는 서울입니다."},
    {"role": "user", "content": "서울 인구는 총 몇 명이야?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



<s> [INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

대한민국의 수도는 어디야? [/INST] 대한민국의 수도는 서울입니다. <|im_end|><s> [INST] 서울 인구는 총 몇 명이야? [/INST]
대한민국  인구조사 결과 에 따르면 , 2020년 서울 시 기준   인구 는 974만 967명입니다입니다 . 하지만 이 숫자는 수시로 변하기 때문에 정확한 인수가 필요하면 통계 청 홈페이지 (www.kostat.go.kr) 또는 관공서에 직접 확인하시는 것이 좋습니다 .<|im_end|>


In [11]:
import pandas as pd
eval_data = pd.read_csv('../data/eval_v5.csv')
texts = []
for i in range(len(eval_data)):
        row = eval_data.iloc[i]
        question = row['질문']
        answer = row['답변']
        item = [
                {"role":"user", "content":question},
                {"role":"assistant", "content":answer},
        ]
        
        text = tokenizer.apply_chat_template(item, tokenize=False, add_generation_prompt=True)
        texts.append(text)

In [14]:
print(texts[0])

<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

알루미늄징크의 단점에 대해 좀 더 알려주실 수 있을까요?포세린 타일의 기대수명은 얼마야? [/INST] 알루미늄징크은 시공이 잘못될 경우 우는 현상이 발생하고 리얼징크보다 가격이 좀 더 비싼 것이 단점입니다.포세린 타일의 기대수명은 50년입니다. <|im_end|>


# fine-tuned model - lora

In [1]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda:2"

#base_model_name = "LDCC/LDCC-SOLAR-10.7B"
#adapter_model_name = "../model/LDCC-SOLAR-10.7B-v5"

base_model_name = "yanolja/EEVE-Korean-10.8B-v1.0"
#adapter_model_name = "../model/yanolja-EEVE-Korean-10.8B-v1.0-v8"
adapter_model_name = "../model/yanolja-EEVE-Korean-10.8B-v1.0-last-default-chat-template"

model = AutoModelForCausalLM.from_pretrained(base_model_name).to(device)
model = PeftModel.from_pretrained(model, adapter_model_name)

tokenizer = AutoTokenizer.from_pretrained(adapter_model_name)

/home/seungbinyang/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/seungbinyang/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/seungbinyang/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Loading checkpoint shards: 100%|██████████| 5/5 [00:03<00:00,  1.50it/s]


In [2]:
text = '도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?'
'''
PROMPT_TEMPLATE = '{sep_token}{question}{eos_token}'
prompt = PROMPT_TEMPLATE.format(question=text,
                                        sep_token=tokenizer.sep_token,
                                        eos_token=tokenizer.eos_token)
        
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
'''
inputs = tokenizer.apply_chat_template(
    [
        {'role':'user','content':text}
    ]
    , add_generation_prompt=True,
    return_tensors='pt'
).to(device)


In [3]:
tokenizer.decode(inputs[0])

'<|im_start|>system\nYou are a helpful assistant.<|im_end|> \n<|im_start|>user\n도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?<|im_end|> \n<|im_start|>assistant\n'

In [4]:
#inputs = tokenizer.encode(f"{tokenizer.sep_token}경질우레탄폼 보온판을 사용한 외단열 시공은 어떤 장점과 단점을 가져?{tokenizer.eos_token}", return_tensors="pt").to(device)

outputs = model.generate(input_ids=inputs, 
                        #num_beams=5,
                         num_beams=1, 
                        eos_token_id=tokenizer.eos_token_id, 
                        max_new_tokens=512)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


<|im_start|>system
You are a helpful assistant. 
<|im_start|>user
도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요? 
<|im_start|>assistant
도배지에 녹이 묻어나오거나 녹이 묻은 손으로 만진 후 도배지에 얼룩이 발생하는 경우 녹오염으로 판단합니다. 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.


In [5]:
def extract_text(input_string):
    index_t = input_string.rfind('assistant')
    if index_t != -1:  
        result = input_string[index_t + len('assistant'):]
    else: 
        raise Exception
    return result.strip()

In [6]:
print(extract_text(tokenizer.decode(outputs[0], skip_special_tokens=True)))

도배지에 녹이 묻어나오거나 녹이 묻은 손으로 만진 후 도배지에 얼룩이 발생하는 경우 녹오염으로 판단합니다. 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.


In [7]:
print(extract_text(tokenizer.decode(outputs[0])))

도배지에 녹이 묻어나오거나 녹이 묻은 손으로 만진 후 도배지에 얼룩이 발생하는 경우 녹오염으로 판단합니다. 녹오염이 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인 : 높은 습기로 인해 도배지 안쪽의 금속의 녹이 도배지에 베어나올 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법 : 제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 누수 원인 : 누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다. 책임소재 : 건물의 소유자나 거주자가 책임이 있습니다. 해결 방법 : 보수작업을 통해 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체해야 합니다. 해당 작업은 개인이 하기 어려운 작업이니 전문가의 도움을 받는 것을 추천합니다.<|im_end|>


In [7]:
def extract_text(input_string):
    index_t = input_string.rfind('<|im_start|>')
    if index_t != -1:  
        result = input_string[index_t + len('<|im_start|>'):]
    else: 
        raise Exception
    return result.strip()


print(extract_text(tokenizer.decode(outputs[0])))

외단열 시공은 동절기의 혹한과 하절기의 혹서에도 건물 내부의 온도와 습도를 안정적으로 유지해주는 장점이 있습니다. 또한, 단열재가 건축물 겉면을 감싸주고 있기 때문에 비가 건물 외벽을 통해 새는 것을 방지하고, 건물 내부의 습도 조절을 쉽게 할 수 있는 등 건축물의 성능과 효율을 향상시키는 데 도움을 줍니다.<|im_end|>


# fine-tuned - full

In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

base_model_name = "../model/GAI-LLM-Yi-Ko-6B-mixed-v15-sft-qlora-full-v4"

model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [ ]:
inputs = tokenizer.encode(f"[INST]방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?[/INST]", return_tensors="pt").to(device)
outputs = model.generate(input_ids=inputs, 
                         max_length=512, 
                         num_beams=5)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# inference

In [10]:
def extract_text(input_string):
    index_t = input_string.find('<|assistant|>')
    if index_t != -1:  
        result = input_string[index_t + len('<|assistant|>'):]
    else: 
        raise Exception
    return result

In [8]:
test = pd.read_csv('../data/test_raw.csv')
test.head(3)

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"


In [11]:
import tqdm

generated_sent = []

for i in tqdm(range(len(test))):
    q = test.iloc[i]['질문']
    prompt = f'<|user|>{q}{tokenizer.eos_token}<|assistant|>'

    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=inputs, max_length=512)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = extract_text(response)

    generated_sent.append(response)

# submission

In [14]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
m = SentenceTransformer('distiluse-base-multilingual-cased-v1')

sub=pd.read_csv('../data/sample_submission.csv')

config_sentence_transformers.json: 100%|██████████| 122/122 [00:00<00:00, 523kB/s]
README.md: 100%|██████████| 2.45k/2.45k [00:00<00:00, 10.2MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 234kB/s]
config.json: 100%|██████████| 556/556 [00:00<00:00, 2.28MB/s]
pytorch_model.bin: 100%|██████████| 539M/539M [00:10<00:00, 50.8MB/s] 
tokenizer_config.json: 100%|██████████| 452/452 [00:00<00:00, 1.64MB/s]
vocab.txt: 100%|██████████| 996k/996k [00:00<00:00, 1.78MB/s]
tokenizer.json: 100%|██████████| 1.96M/1.96M [00:00<00:00, 2.10MB/s]
2_Dense/config.json: 100%|██████████| 114/114 [00:00<00:00, 405kB/s]
pytorch_model.bin: 100%|██████████| 1.58M/1.58M [00:01<00:00, 1.40MB/s]


In [15]:
encode_list=[]
for i in range(len(generated_sent)):
  embed=m.encode(generated_sent[i]) #주어진 모델로 인코딩
  encode_list.append(embed)

In [ ]:
for i in range(len(encode_list)):
  sub.loc[i, 'vec_0':'vec_511']=encode_list[i] #제출 파일에 끼워넣기

In [17]:
sub.head(2)

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.044647,0.044686,0.019709,0.003137,0.059527,0.014705,-0.013557,0.025025,0.026930,...,0.018252,-0.041767,-0.011106,-0.032668,-0.028045,0.010286,0.053333,0.019077,0.004254,0.030388
1,TEST_001,0.001539,-0.014106,0.006531,0.013342,0.069398,-0.013717,-0.008829,-0.023080,-0.008041,...,-0.025134,-0.016303,0.043635,-0.045904,-0.025309,0.035036,-0.005466,-0.009464,0.036712,-0.004162


In [19]:
sub.set_index('id',inplace=True)

In [20]:
sub.head(2)

,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
id,,,,,,,,,,,,,,,,,,,,,
TEST_000,0.044647,0.044686,0.019709,0.003137,0.059527,0.014705,-0.013557,0.025025,0.026930,0.023581,...,0.018252,-0.041767,-0.011106,-0.032668,-0.028045,0.010286,0.053333,0.019077,0.004254,0.030388
TEST_001,0.001539,-0.014106,0.006531,0.013342,0.069398,-0.013717,-0.008829,-0.023080,-0.008041,0.016860,...,-0.025134,-0.016303,0.043635,-0.045904,-0.025309,0.035036,-0.005466,-0.009464,0.036712,-0.004162


In [21]:
sub.to_csv('../result/GAI-LLM-Yi-Ko-6B-mixed-v15-qlora-v1.csv')

In [33]:
with open('../result/GAI-LLM-Yi-Ko-6B-mixed-v15-qlora-v1.txt ', 'w+') as file:
    file.write('\n'.join(generated_sent))